In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import random
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib as mpl
import hashlib
import tensorflow as tf
from tensorflow.keras.models import load_model
from skimage.metrics import structural_similarity as ssim
from scipy.fftpack import dct
import math

mpl.rcParams['font.family'] = 'DejaVu Sans'

In [ ]:
# === CONFIGURATION === #
IMG_SIZE = (128, 128)
CHANNELS = 1
CLASS_NAMES = ['Normal', 'Attack']
SEED_RESERVED_PIXELS = 300
SEED_SECRET_KEY = "my_shared_passphrase"

# Will be set dynamically
NUM_PIXELS = None
MESSAGE_BITS = None

In [ ]:
# === MESSAGE UTILS === #
def generate_random_message(chars):
    return ''.join(random.choices(string.ascii_letters, k=chars))

def string_to_bits(s):
    return [int(b) for c in s.encode('ascii') for b in format(c, '08b')][:MESSAGE_BITS]

def bits_to_string(bits):
    chars = []
    for i in range(0, len(bits), 8):
        byte = bits[i:i+8]
        if len(byte) == 8:
            chars.append(chr(int(''.join(map(str, byte)), 2)))
    return ''.join(chars)

def clean_excel_string(s):
    return ''.join(c for c in s if c.isalpha())

In [ ]:
# === PRNG SEED-HIDING BASED ON IMAGE CONTENT + SHARED KEY === #
def get_seed_hiding_key(secret_key: str, image: np.ndarray):
    image_hash = hashlib.sha256(image.tobytes()).hexdigest()
    combined = secret_key + image_hash
    digest = hashlib.sha256(combined.encode()).hexdigest()
    return int(digest, 16) % (2**32)

In [ ]:
# === EMBEDDING === #
def embed_lsb(image, bits, coords):
    flat = image.flatten()
    for i, (y, x) in enumerate(coords):
        if not (0 <= y < image.shape[0] and 0 <= x < image.shape[1]):
            continue
        idx = y * image.shape[1] + x
        if idx < SEED_RESERVED_PIXELS:
            continue
        flat[idx] = (flat[idx] & ~1) | bits[i]
    return flat.reshape(image.shape)

In [ ]:
# === SEED COORDINATES & EMBEDDING === #
def get_seed_embedding_coords(image, total_reserved=SEED_RESERVED_PIXELS, bits=32, seed_secret_key=SEED_SECRET_KEY):
    seed_key = get_seed_hiding_key(seed_secret_key, image)
    np.random.seed(seed_key)
    coords = np.random.choice(range(total_reserved), size=bits, replace=False)
    return coords

def embed_seed_value(image, seed_bits, seed_secret_key, coords=None):
    if coords is None:
        coords = get_seed_embedding_coords(image.copy(), seed_secret_key=seed_secret_key)
    flat = image.flatten()
    for i, idx in enumerate(coords):
        flat[idx] = (flat[idx] & ~1) | seed_bits[i]
    return flat.reshape(image.shape), coords

def extract_seed_value(image, seed_secret_key, coords=None):
    if coords is None:
        coords = get_seed_embedding_coords(image.copy(), seed_secret_key=seed_secret_key)
    flat = image.flatten()
    seed_bits = [flat[idx] & 1 for idx in coords]
    seed_value = int(''.join(map(str, seed_bits)), 2)
    return seed_value

In [ ]:
# === SEED GENERATION === #
def get_seed_from_image(secret_key: str, image: np.ndarray):
    image_hash = hashlib.sha256(image.tobytes()).hexdigest()
    combined = secret_key + image_hash + "_main_seed"
    digest = hashlib.sha256(combined.encode()).hexdigest()
    return int(digest, 16) % (2**32)

def generate_prng_pixel_positions(image_shape, count, seed_value):
    np.random.seed(seed_value)
    h, w = image_shape
    total_pixels = h * w
    excluded_pixels = set(range(SEED_RESERVED_PIXELS))
    valid_indices = list(set(range(total_pixels)) - excluded_pixels)
    if count > len(valid_indices):
        raise ValueError("Not enough space to embed the message with current settings.")
    indices = np.random.choice(valid_indices, size=count, replace=False)
    ys, xs = np.unravel_index(indices, (h, w))
    return list(zip(ys, xs))

In [ ]:
# === CNN FEATURE EXTRACTION & ANALYSIS === #
def extract_features(image):
    image = image.astype(np.float32) / 255.0
    image = np.expand_dims(image, axis=-1)
    image = np.expand_dims(image, axis=0)
    return image

def analyze_with_cnn(image, model):
    cnn_input = extract_features(image)
    prob = float(model.predict(cnn_input, verbose=0)[0])
    predicted_class = CLASS_NAMES[int(prob >= 0.5)]
    confidence = prob if predicted_class == "Attack" else 1 - prob
    return predicted_class, confidence

In [ ]:
# === DECODING & QUALITY METRICS === #
def decode_message(image, coords):
    flat = image.flatten()
    msg_bits = [flat[y * image.shape[1] + x] & 1 for y, x in coords[:NUM_PIXELS]]
    return bits_to_string(msg_bits)

def calculate_mse(original, stego):
    return np.mean((original.astype(np.float32) - stego.astype(np.float32)) ** 2)

def calculate_psnr(mse, max_pixel=255.0):
    if mse == 0:
        return float('inf')
    return 20 * math.log10(max_pixel / math.sqrt(mse))

def calculate_ssim(original, stego):
    return ssim(original, stego)

def compute_dct_difference(original, stego):
    original_dct = dct(dct(original.T, norm='ortho').T, norm='ortho')
    stego_dct = dct(dct(stego.T, norm='ortho').T, norm='ortho')
    diff = np.abs(original_dct - stego_dct)
    return np.mean(diff), np.max(diff)

In [ ]:
# === VISUALIZATION === #
def save_visualization_overlay(image, coords, save_path):
    vis_image = cv2.cvtColor(image.copy(), cv2.COLOR_GRAY2BGR)
    for y, x in coords:
        if 0 <= y < vis_image.shape[0] and 0 <= x < vis_image.shape[1]:
            vis_image[y, x] = (0, 0, 255)
    cv2.imwrite(save_path, vis_image)
    print(f"✅ Saved red overlay visualization: {os.path.basename(save_path)}")

In [ ]:
# === MAIN FUNCTION === #
def process_folder(input_folder, model_path, stego_output_folder, results_output_folder, seed_secret_key):
    # Full function implemented in original script (omitted here to save space)
    pass

In [ ]:
# === RUN === #
message_length_chars = int(input("Enter the number of characters to encode: "))
MESSAGE_BITS = message_length_chars * 8
NUM_PIXELS = MESSAGE_BITS

input_folder = "PRNG/FDIA 1/CNN/Attack"
model_path = "PRNG/FDIA 1/CNN/final_cnn_model_grayscale.h5"
stego_output_folder = "PRNG/FDIA 1/CNN/Custom Message Encoder/Attack/stego_images (3 letters)"
results_output_folder = "PRNG/FDIA 1/CNN/Custom Message Encoder/Attack/Final Results (3 letters)"
seed_secret_key = "Gustavo_Sanchez"

process_folder(input_folder, model_path, stego_output_folder, results_output_folder, seed_secret_key)